In [ ]:
import sys
sys.path.append("../lib")
import os
from influxdb import InfluxDBClient, DataFrameClient
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
from pyutils import *
from pdutils import *
from pdpltutils import *



In [ ]:
# Hardcode cloudlet IP and port for DB
CLOUDLET_IP = '128.2.208.248'
EPC_IP = '192.168.25.4'
UE_IP = '192.168.25.2'
CLOUDLET_PORT = 8086

IPMAP = {CLOUDLET_IP:"CLOUDLET",EPC_IP:"EPC",UE_IP:"UE"}

# ICMP databases
CLOUDLET_ICMP_DB = 'cloudleticmp'
WATERSPOUT_ICMP_DB = 'waterspouticmp'
UE_ICMP_DB = 'ueicmp'
XRAN_ICMP_DB = 'xranicmp'
SEG_DB = 'segmentation'
SEG_DB = 'segmentation'

### Functions

In [ ]:
def dumpdf(df, rows=5):
    print(df.shape)
    print(df.head(rows))
    print(df.dtypes)

### Clients

In [ ]:
''' Node clients '''
cloudlet_icmp_client = InfluxDBClient(host=CLOUDLET_IP, port=CLOUDLET_PORT, database=CLOUDLET_ICMP_DB)
df_cloudlet_icmp_client = DataFrameClient(host=CLOUDLET_IP, port=CLOUDLET_PORT, database=CLOUDLET_ICMP_DB)

waterspout_icmp_client = InfluxDBClient(host=CLOUDLET_IP, port=CLOUDLET_PORT, database=WATERSPOUT_ICMP_DB)
df_waterspout_icmp_client = DataFrameClient(host=CLOUDLET_IP, port=CLOUDLET_PORT, database=WATERSPOUT_ICMP_DB)

ue_icmp_client = InfluxDBClient(host=CLOUDLET_IP, port=CLOUDLET_PORT, database=UE_ICMP_DB)
df_ue_icmp_client = DataFrameClient(host=CLOUDLET_IP, port=CLOUDLET_PORT, database=UE_ICMP_DB)

xran_icmp_client = InfluxDBClient(host=CLOUDLET_IP, port=CLOUDLET_PORT, database=XRAN_ICMP_DB)
df_xran_icmp_client = DataFrameClient(host=CLOUDLET_IP, port=CLOUDLET_PORT, database=XRAN_ICMP_DB)

measure = 'latency'

In [ ]:
''' Segmentation Clients '''
seg_client = InfluxDBClient(host=CLOUDLET_IP, port=CLOUDLET_PORT, database=SEG_DB)
df_seg_client = DataFrameClient(host=CLOUDLET_IP, port=CLOUDLET_PORT, database=SEG_DB)

measure = 'uplink'
seg_ul_df = df_seg_client.query("select * from {}".format(measure))[measure]
measure = 'downlink'
seg_dl_df = df_seg_client.query("select * from {}".format(measure))[measure]
seqlst = list(set(seg_ul_df.sequence).intersection(seg_dl_df.sequence))
seg_df = pd.concat([seg_ul_df,seg_dl_df])
seg_df = seg_df[seg_df.sequence.isin(seqlst)].sort_values(['sequence','direction'],ascending=[True,False]).drop_duplicates()

dumpdf(seg_df)

### Queries

In [ ]:
''' Node queries '''
measure = 'latency'
cloudlet_icmp_df = df_cloudlet_icmp_client.query("select * from {}".format(measure))[measure]
waterspout_icmp_df = df_waterspout_icmp_client.query("select * from {}".format(measure))[measure]
ue_icmp_df = df_ue_icmp_client.query("select * from {}".format(measure))[measure]
xran_icmp_df = df_xran_icmp_client.query("select * from {}".format(measure))[measure]

In [ ]:
''' Segmentation queries '''
measure = 'uplink'
seg_ul_df = df_seg_client.query("select * from {}".format(measure))[measure]
measure = 'downlink'
seg_dl_df = df_seg_client.query("select * from {}".format(measure))[measure]
seqlst = list(set(seg_ul_df.sequence).intersection(seg_dl_df.sequence))
seg_df = pd.concat([seg_ul_df,seg_dl_df])
seg_df = seg_df[seg_df.sequence.isin(seqlst)].sort_values(['sequence','direction'],ascending=[True,False]).drop_duplicates()


In [ ]:
display_all(seg_df)

In [ ]:
tdfx = seg_df.copy()
tdfx['TIMESTAMP'] = tdfx.start.map(lambda epoch: pd.to_datetime(epoch, unit = 's'))
dumpdf(tdfx)

### Inspection

In [ ]:
''' Get list of sequences in all three dataframes '''
seqminset = list(set(ue_icmp_df.sequence) \
    .intersection((set(cloudlet_icmp_df.sequence) \
    .intersection(set(waterspout_icmp_df.sequence)))))

print(seqminset)

In [ ]:
''' Waterspout fix '''
tdfx = waterspout_icmp_df.copy()
tdfx['SRCNAME'] = tdfx.src.map(lambda col: IPMAP[col])
tdfx['DSTNAME'] = tdfx.dst.map(lambda col: IPMAP[col])
# dumpdf(tdfx[tdfx.sequence.isin(seqminset)],rows=7)
dumpdf(tdfx[tdfx.sequence.isin(seqminset)][['sequence','src','dst','SRCNAME','DSTNAME']],rows=7)

thresh = 0.1
tdfy = tdfx[(tdfx.SRCNAME == "UE") & (tdfx.DSTNAME == 'EPC')]

tdfy['DELTA'] = tdfy.epoch - tdfy.epoch.shift(1)
tdfy['DELTA'] = tdfy.DELTA.map(lambda col: col if col < thresh else np.NAN)
tdfy = tdfy.dropna(subset=['DELTA'])
dumpdf(tdfy)
print(tdfy.DELTA.median())

tdfz = tdfx.copy().groupby(by=['sequence','src','dst']).max().sort_values('epoch').reset_index()
dumpdf(tdfz)

In [ ]:
''' Segmentation inspection '''
tdfx = seg_df.copy()
dumpdf(tdfx)

In [ ]:
''' EPC to Cloudlet '''
wdf = waterspout_icmp_df.copy()
cdf = cloudlet_icmp_df.copy()
seqminset = list(set(waterspout_icmp_df.sequence) \
    .intersection((set(cloudlet_icmp_df.sequence))))
# print(seqminset)
wdf['NAME'] = 'waterspout'
cdf['NAME'] = 'cloudlet'
jdf = pd.concat([wdf,cdf])
jdf = jdf[jdf.sequence.isin(seqminset)]
jdf = drp_lst(jdf,['data_time','identifier','htime'])
jdf['sepoch'] = jdf.epoch.map(lambda col: "{}".format(col))
# dumpdf(wdf)
# dumpdf(cdf)
display_all(jdf.sort_values('epoch')[['NAME','src','dst','sequence','sepoch']])

### XRAN inspection

In [ ]:
waterspout_icmp_df = df_waterspout_icmp_client.query("select * from {}".format(measure))[measure]

In [ ]:
tdfx = epochToTS(xran_icmp_df.copy().reset_index(drop=True))
tdfy = epochToTS(ue_icmp_df.copy().reset_index(drop=True))
tdfz = epochToTS(waterspout_icmp_df.copy().reset_index(drop=True))
tdfw = epochToTS(cloudlet_icmp_df.copy().reset_index(drop=True))
dumpdf(tdfx)

In [ ]:
''' Use 6/9/2022 Data '''
tdfx = tdfx[tdfx.TIMESTAMP.dt.date >= datetime.date.today()]
tdfy = tdfy[tdfy.TIMESTAMP.dt.date >= datetime.date.today()]
tdfz = tdfz[tdfz.TIMESTAMP.dt.date >= datetime.date.today()]
tdfw = tdfw[tdfw.TIMESTAMP.dt.date >= datetime.date.today()]

In [ ]:
# tdfz = tdfz[tdfz.TIMESTAMP.dt.date >= datetime.date.today()]
dumpdf(tdfz)

In [ ]:
''' Get list of sequences in all three dataframes '''
seqminset = list(set(tdfy.sequence) \
    .intersection((set(tdfz.sequence) \
    .intersection(set(tdfw.sequence)))))

seqminset.sort()
# print(seqminset)
sseqminset = list(set(seqminset).intersection(set(tdfx.sequence)))
sseqminset.sort()
print(sseqminset)


In [ ]:
print(list(tdfx[tdfx.sequence.isin(seqminset)].sequence.drop_duplicates().sort_values()))

In [ ]:
seq = '161/41216'

dumpdf(tdfx[tdfx.sequence == seq])
dumpdf(tdfz[tdfz.sequence == seq])

#     .intersection((set(cloudlet_icmp_df.sequence) \
#     .intersection(set(waterspout_icmp_df.sequence)))))

In [ ]:
tdfx[tdfx.sequence == seq].TIMESTAMP.iloc[0] - tdfz[tdfz.sequence == seq].TIMESTAMP.iloc[0]


In [ ]:
seqminset.sort()
print(seqminset)